In [130]:
import numpy as np
data = np.genfromtxt("data/data.csv",delimiter=',',skip_header=1)

In [131]:
theta_old = (0,0)

In [132]:
def init_theta():
    return (0,0)

def update_theta(theta_old,data,learning_rate):
    prices = data[:,1]
    mileages = data[:,0]
    estimates = theta_old[0]+theta_old[1]*mileages
    m = prices.size
    theta0_new = theta_old[0]-learning_rate*sum(estimates-prices)/m
    theta1_new = theta_old[1]-learning_rate*sum((estimates-prices)*mileages)/m
    return (theta0_new,theta1_new)

In [152]:
theta_test = init_theta()
theta_test = update_theta(theta_test,data,0.5)

In [159]:
theta_test = update_theta(theta_test,data,0.5)
print(theta_test)

(-1.0474004077805004e+72, -1.334130633028402e+77)


In [150]:
prices = data[:,1]
mileages = data[:,0]
estimates = theta[0]+theta[1]*mileages
m = prices.size
theta0_new = theta_old[0]-0.5*sum(estimates-prices)/m
theta1_new = theta_old[1]-0.5*sum((estimates-prices)*mileages)/m

In [151]:
theta1_new

-12581954970.0

In [100]:
mileages

array([240000., 139800., 150500., 185530., 176000., 114800., 166800.,
        89000., 144500.,  84000.,  82029.,  63060.,  74000.,  97500.,
        67000.,  76025.,  48235.,  93000.,  60949.,  65674.,  54000.,
        68500.,  22899.,  61789.])

In [124]:
m

24